In [1]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
# plt.style.use('seaborn-pastel')

from brian2 import pA, Hz, second, ms
import numpy as np

from results import Results
from plot import (MidpointNormalize, weight_matrix, remove_axes_less, prune_figure, prune_figure_less, prune_figure_more,
                  plot_spike_raster, plot_firing_rate, plot_voltage_detail)

from generate import weight_matrix_from_flat_inds_weights, adjacency_matrix_from_flat_inds, generate_adjacency_indices_within
import pandas as pd
from sklearn.neighbors import KernelDensity
def cv(a):
    return a.std() / a.mean()

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 14562 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [391]:
plt.close('all')

In [3]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/'
# targetPath = 'C:/Users/mikejseay/Documents/BrianResultsHoff/'

In [4]:
targetFile1 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_guessGoodWeights2e3p025LogNormal_movAvg1_2021-08-05-11-02_results'  # converges at 330+
targetFile2 = 'classicJercog_2000_1_cross-homeo-pre-scalar-homeo_guessGoodWeights2e3p1LogNormal_fullConn_2021-08-13-09-38_results'  # converges at 230+

In [5]:
R1 = Results()
R1.init_from_file(targetFile1, targetPath)

if R1.p['useRule'][:5] == 'cross':
    R1.p['alpha2'] = -1e-12
    R1.p['tauPlasticityTrials'] = None
elif R1.p['useRule'] == 'homeo':
    R1.p['alpha2'] = -1e-12
    R1.p['tauPlasticityTrials'] = None

if 'wEEScale' not in R1.p:
    R1.p['wEEScale'] = None
    R1.p['wIEScale'] = None
    R1.p['wEIScale'] = None
    R1.p['wIIScale'] = None

importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a1={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R1.rID,
    R1.p['wEEScale'],R1.p['wIEScale'],R1.p['wEIScale'],R1.p['wIIScale'],
    R1.p['tauUpFRTrials'], R1.p['alpha1'] / pA, R1.p['alpha2'] / pA, R1.p['tauPlasticityTrials'])

wEEInit1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nExc'], R1.preEE, R1.posEE, R1.wEE_init)
wIEInit1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nInh'], R1.preIE, R1.posIE, R1.wIE_init)
wEIInit1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nExc'], R1.preEI, R1.posEI, R1.wEI_init)
wIIInit1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nInh'], R1.preII, R1.posII, R1.wII_init)

wEEFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nExc'], R1.preEE, R1.posEE, R1.wEE_final)
wIEFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nInh'], R1.preIE, R1.posIE, R1.wIE_final)
wEIFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nExc'], R1.preEI, R1.posEI, R1.wEI_final)
wIIFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nInh'], R1.preII, R1.posII, R1.wII_final)

wFullInit1 = np.block([[wEEInit1, wIEInit1], [-wEIInit1, -wIIInit1]])
wFullFinal1 = np.block([[wEEFinal1, wIEFinal1], [-wEIFinal1, -wIIFinal1]])

# get summed presynaptic input to each unit

wEEInitP1 = np.nansum(wEEInit1, 0)
wIEInitP1 = np.nansum(wIEInit1, 0)
wEIInitP1 = np.nansum(wEIInit1, 0)
wIIInitP1 = np.nansum(wIIInit1, 0)

wEEFinalP1 = np.nansum(wEEFinal1, 0)
wIEFinalP1 = np.nansum(wIEFinal1, 0)
wEIFinalP1 = np.nansum(wEIFinal1, 0)
wIIFinalP1 = np.nansum(wIIFinal1, 0)

print('wEE mult:', R1.wEE_final.mean() / R1.wEE_init.mean())
print('wIE mult:', R1.wIE_final.mean() / R1.wIE_init.mean())
print('wEI mult:', R1.wEI_final.mean() / R1.wEI_init.mean())
print('wII mult:', R1.wII_final.mean() / R1.wII_init.mean())

wEE mult: 0.9607961654663084
wIE mult: 0.9257208082411026
wEI mult: 1.046995162963867
wII mult: 0.8950419249358


In [6]:
R2 = Results()
R2.init_from_file(targetFile2, targetPath)

if R2.p['useRule'][:5] == 'cross':
    R2.p['alpha2'] = -1e-12
    R2.p['tauPlasticityTrials'] = None
elif R2.p['useRule'] == 'homeo':
    R2.p['alpha2'] = -1e-12
    R2.p['tauPlasticityTrials'] = None

if 'wEEScale' not in R2.p:
    R2.p['wEEScale'] = None
    R2.p['wIEScale'] = None
    R2.p['wEIScale'] = None
    R2.p['wIIScale'] = None

importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a2={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R2.rID,
    R2.p['wEEScale'],R2.p['wIEScale'],R2.p['wEIScale'],R2.p['wIIScale'],
    R2.p['tauUpFRTrials'], R2.p['alpha2'] / pA, R2.p['alpha2'] / pA, R2.p['tauPlasticityTrials'])

wEEInit2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nExc'], R2.preEE, R2.posEE, R2.wEE_init)
wIEInit2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nInh'], R2.preIE, R2.posIE, R2.wIE_init)
wEIInit2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nExc'], R2.preEI, R2.posEI, R2.wEI_init)
wIIInit2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nInh'], R2.preII, R2.posII, R2.wII_init)

wEEFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nExc'], R2.preEE, R2.posEE, R2.wEE_final)
wIEFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nInh'], R2.preIE, R2.posIE, R2.wIE_final)
wEIFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nExc'], R2.preEI, R2.posEI, R2.wEI_final)
wIIFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nInh'], R2.preII, R2.posII, R2.wII_final)

wFullInit2 = np.block([[wEEInit2, wIEInit2], [-wEIInit2, -wIIInit2]])
wFullFinal2 = np.block([[wEEFinal2, wIEFinal2], [-wEIFinal2, -wIIFinal2]])

# get summed presynaptic input to each unit

wEEInitP2 = np.nansum(wEEInit2, 0)
wIEInitP2 = np.nansum(wIEInit2, 0)
wEIInitP2 = np.nansum(wEIInit2, 0)
wIIInitP2 = np.nansum(wIIInit2, 0)

wEEFinalP2 = np.nansum(wEEFinal2, 0)
wIEFinalP2 = np.nansum(wIEFinal2, 0)
wEIFinalP2 = np.nansum(wEIFinal2, 0)
wIIFinalP2 = np.nansum(wIIFinal2, 0)

print('wEE mult:', R2.wEE_final.mean() / R2.wEE_init.mean())
print('wIE mult:', R2.wIE_final.mean() / R2.wIE_init.mean())
print('wEI mult:', R2.wEI_final.mean() / R2.wEI_init.mean())
print('wII mult:', R2.wII_final.mean() / R2.wII_init.mean())

wEE mult: 1.0086468378702806
wIE mult: 1.0044846428765195
wEI mult: 0.987797370323768
wII mult: 0.9804810418023001


In [7]:
# let's set up an artificial learning scenario
# we will start with weight matrices from the "full connectivity"
# we will also start with some post-convergence trial position
# we will take the individual unit firing rates at that position
# then we will calculate the proposed weight changes given each interpretation of the matrices

In [7]:
p = R1.p

In [8]:
plt.close('all')

In [9]:
# save the difference in the mean between the full and sparse II mat for each trial
# in 3 scenarios:
# 1: fromSim: the actual observed individual unit firing rates
# 2: randn1: normally distributed firing rates near the set point
# 3: randn2: normally distibuted but biased to be low

dwIIFullMeanTrial1 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial1 = np.empty(R1.trialUpFRExc.size)
dwIIFullMeanTrial2 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial2 = np.empty(R1.trialUpFRExc.size)
dwIIFullMeanTrial3 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial3 = np.empty(R1.trialUpFRExc.size)

commonRand = np.random.normal(0, 0.05, R1.trialUpFRExc.size)
dummyInd = 0

for trialInd in range(R1.trialUpFRExc.size):
    movAvgUpFRInhUnits1 = R1.trialUpFRInhUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits2 = np.random.normal(1, 0.1, p['nInh']) * p['setUpFRInh']
    movAvgUpFRInhUnits3 = p['setUpFRInh'] + (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dwIIHCurrent1 = -np.outer(movAvgUpFRInhUnits1, (p['setUpFRInh'] - movAvgUpFRInhUnits1))
    dwIIHCurrent2 = -np.outer(movAvgUpFRInhUnits2, (p['setUpFRInh'] - movAvgUpFRInhUnits2))
    dwIIHCurrent3 = -np.outer(movAvgUpFRInhUnits3, (p['setUpFRInh'] - movAvgUpFRInhUnits3))
    dwIIFullMeanTrial1[trialInd] = dwIIHCurrent1.mean()
    dwIISparseMeanTrial1[trialInd] = dwIIHCurrent1[R1.preII, R1.posII].mean()
    dwIIFullMeanTrial2[trialInd] = dwIIHCurrent2.mean()
    dwIISparseMeanTrial2[trialInd] = dwIIHCurrent2[R1.preII, R1.posII].mean()
    dwIIFullMeanTrial3[trialInd] = dwIIHCurrent3.mean()
    dwIISparseMeanTrial3[trialInd] = dwIIHCurrent3[R1.preII, R1.posII].mean()
    dummyInd += 1

dwIIFullSparseDiffTrials1 = dwIISparseMeanTrial1 - dwIIFullMeanTrial1
dwIIFullSparseDiffTrials2 = dwIISparseMeanTrial2 - dwIIFullMeanTrial2
dwIIFullSparseDiffTrials3 = dwIISparseMeanTrial3 - dwIIFullMeanTrial3

In [10]:
inhDF = pd.DataFrame({'fromSim': movAvgUpFRInhUnits1,
                      'randn1': movAvgUpFRInhUnits2,
                      'randn2': movAvgUpFRInhUnits3})

In [11]:
inhDF.describe()

,fromSim,randn1,randn2
count,400.000000,400.000000,400.000000
mean,14.033333,13.995539,13.455829
std,2.987182,1.478974,1.464156
min,5.925926,9.580195,8.356762
25%,11.851852,13.024114,12.443207
50%,14.074074,13.894081,13.443605
75%,16.296296,15.017009,14.451009
max,27.407407,19.559547,18.063289


In [18]:
# plotting the mean dWII for each trial for the full matrix, the sparse matrix, and the average difference
# for all 3 scenarios

f, ax = plt.subplots(3, 1, sharey=True)

ax[0].hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax[0].plot(dwIIFullMeanTrial1, alpha=0.5, color='b')
ax[0].plot(dwIISparseMeanTrial1, alpha=0.5, color='g')
ax[0].hlines(dwIIFullSparseDiffTrials1.mean(), 0, R1.trialUpFRExc.size)

ax[1].hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax[1].plot(dwIIFullMeanTrial2, alpha=0.5, color='b')
ax[1].plot(dwIISparseMeanTrial2, alpha=0.5, color='g')
ax[1].hlines(dwIIFullSparseDiffTrials2.mean(), 0, R1.trialUpFRExc.size)

ax[2].hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax[2].plot(dwIIFullMeanTrial3, alpha=0.5, color='b')
ax[2].plot(dwIISparseMeanTrial3, alpha=0.5, color='g')
ax[2].hlines(dwIIFullSparseDiffTrials3.mean(), 0, R1.trialUpFRExc.size)

In [19]:
# goal here is to actually calculate the dw on EVERY trial (massive array)
# to save memory we will accumulate a sum and divide at the end

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSum = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSum = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSum = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSum = np.zeros((p['nInh'], p['nInh']))

trialDiffII = np.empty(())

for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = R1.trialUpFRExcUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits = R1.trialUpFRInhUnits[trialInd, :] * Hz
    dwEEHTrialSum += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSum += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSum += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSum += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

dwEEHHat = dwEEHTrialSum / nTrials
dwEIHHat = dwEIHTrialSum / nTrials
dwIEHHat = dwIEHTrialSum / nTrials
dwIIHHat = dwIIHTrialSum / nTrials

dwEEHHatS = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatS = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatS = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatS = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatS[R1.preEE, R1.posEE] = dwEEHHat[R1.preEE, R1.posEE]
dwEIHHatS[R1.preEI, R1.posEI] = dwEIHHat[R1.preEI, R1.posEI]
dwIEHHatS[R1.preIE, R1.posIE] = dwIEHHat[R1.preIE, R1.posIE]
dwIIHHatS[R1.preII, R1.posII] = dwIIHHat[R1.preII, R1.posII]

In [177]:
dwIIHHatSInv = np.full((p['nInh'], p['nInh']), np.nan)
dwIIHHatSInv[R1.posII, R1.preII] = dwIIHHat[R1.posII, R1.preII]

In [179]:
dwIIHHatSAnti = dwIIHHat.copy()
dwIIHHatSAnti[R1.preII, R1.posII] = np.nan

In [129]:
print(np.nanmean(dwEEHHatS) - dwEEHHat.mean())
print(np.nanmean(dwEIHHatS) - dwEIHHat.mean())
print(np.nanmean(dwIEHHatS) - dwIEHHat.mean())
print(np.nanmean(dwIIHHatS) - dwIIHHat.mean())

-0.022719073430059446
-0.12391167628929009
-0.07269817233591887
-0.2797260996483352


In [21]:
# now let's repeat the same thing but draw the firing rates from a normal distribution centered around the setpoints
# and see if the bias is still there...

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSumF = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSumF = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSumF = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSumF = np.zeros((p['nInh'], p['nInh']))

for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = np.random.normal(1, 0.1, p['nExc']) * p['setUpFRExc']
    movAvgUpFRInhUnits = np.random.normal(1, 0.1, p['nInh']) * p['setUpFRInh']
    dwEEHTrialSumF += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSumF += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSumF += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSumF += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

dwEEHHatF = dwEEHTrialSumF / nTrials
dwEIHHatF = dwEIHTrialSumF / nTrials
dwIEHHatF = dwIEHTrialSumF / nTrials
dwIIHHatF = dwIIHTrialSumF / nTrials

dwEEHHatSF = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatSF = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatSF = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatSF = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatSF[R1.preEE, R1.posEE] = dwEEHHatF[R1.preEE, R1.posEE]
dwEIHHatSF[R1.preEI, R1.posEI] = dwEIHHatF[R1.preEI, R1.posEI]
dwIEHHatSF[R1.preIE, R1.posIE] = dwIEHHatF[R1.preIE, R1.posIE]
dwIIHHatSF[R1.preII, R1.posII] = dwIIHHatF[R1.preII, R1.posII]

In [22]:
f, ax = plt.subplots(1, 2)
ax[0].hist(R1.preII, bins=400)
ax[1].hist(R1.posII, bins=400)

(array([ 87.,  90.,  94.,  94., 106.,  99., 109., 103., 105.,  83.,  89.,
        102., 107., 101.,  90.,  95.,  97., 114., 103., 109.,  93.,  89.,
         97.,  91., 107., 102.,  91.,  81.,  95.,  99.,  86., 108.,  99.,
         95., 110.,  98., 114.,  92., 108., 103., 102., 106.,  96., 113.,
         91., 100.,  92., 103.,  86., 100., 109.,  82.,  89., 104., 100.,
         98.,  95., 108.,  94.,  99.,  99., 101., 108., 109., 107., 125.,
        118.,  96., 112.,  99.,  97.,  84., 102.,  93., 105.,  87.,  98.,
         94.,  87., 103., 109., 120.,  99.,  99., 100., 106.,  86., 111.,
        105., 107., 126., 113., 102.,  86.,  90., 101.,  97.,  95., 104.,
         88., 109., 112.,  95.,  93., 112., 106., 100., 108., 110., 100.,
         98., 108.,  99., 106.,  94.,  87.,  99.,  95., 101., 103.,  90.,
        104.,  91.,  99.,  98., 107., 110., 105.,  97.,  98.,  86., 117.,
         88., 104.,  92., 101.,  89., 109., 109.,  93.,  91., 105., 106.,
         94., 103.,  88., 106., 115., 

In [23]:
print(np.nanmean(dwEEHHatSF) - dwEEHHatF.mean())
print(np.nanmean(dwEIHHatSF) - dwEIHHatF.mean())
print(np.nanmean(dwIEHHatSF) - dwIEHHatF.mean())
print(np.nanmean(dwIIHHatSF) - dwIIHHatF.mean())

0.00013743233586260687
0.00048803256966801546
-0.00014300326783331027
-0.005680311537440637


In [378]:
plt.close('all')

In [188]:
# maybe the distribution of the Inh FRs from the training are not normal?

movAvgUpFRInhUnits2 = np.random.normal(1, 0.1, p['nInh']) * p['setUpFRInh']
movAvgUpFRInhUnits3 = p['setUpFRInh'] + (commonRand[0] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']

f, ax = plt.subplots()
ax.hist(R1.trialUpFRInhUnits[startTrialInd:endTrialInd].ravel(), bins=40, density=True, histtype='step', color='red')
ax.hist(movAvgUpFRInhUnits2 / Hz, bins=40, density=True, histtype='step', color='blue')
ax.hist(movAvgUpFRInhUnits3 / Hz, bins=40, density=True, histtype='step', color='green')
ax.vlines(p['setUpFRInh'] / Hz, *ax.get_ylim(), linestyle='--')
ax.set(xlabel='I FR (Hz)', ylabel='Proportion of occurences')
ax.legend(('actual', 'fake1', 'fake2'))

In [260]:
frCorrIIBetweenTrials = np.corrcoef(R1.trialUpFRInhUnits[startTrialInd:endTrialInd])
frCorrIIBetweenUnits = np.corrcoef(R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T)
frCorrIIBetweenTrials[np.triu_indices(frCorrIIBetweenTrials.shape[0])] = np.nan
frCorrIIBetweenUnits[np.triu_indices(frCorrIIBetweenUnits.shape[0])] = np.nan

In [261]:
frCorrIIBetweenUnitsAnti = frCorrIIBetweenUnits.copy()
frCorrIIBetweenUnitsAnti[R1.preII, R1.posII] = np.nan
print(np.nanmean(frCorrIIBetweenUnits))
print(np.nanmean(frCorrIIBetweenUnits[R1.preII, R1.posII]))
print(np.nanmean(frCorrIIBetweenUnitsAnti))

-1.06058027142493e-05
-0.022405479591587556
0.00743690044267962


In [262]:
# does the same type of bias exist in the correlations between FRs???

frCorrIIBetweenUnitsS = np.full(frCorrIIBetweenUnits.shape, np.nan)
frCorrIIBetweenUnitsS[R1.preII, R1.posII] = frCorrIIBetweenUnits[R1.preII, R1.posII]

allNumbers = frCorrIIBetweenUnits.ravel()

vlims = (-np.nanmax(np.fabs(frCorrIIBetweenUnits)), np.nanmax(np.fabs(frCorrIIBetweenUnits)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 7))
weight_matrix(ax7[0], frCorrIIBetweenUnitsS, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], frCorrIIBetweenUnits, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)

In [265]:
frCorrEEBetweenTrials = np.corrcoef(R1.trialUpFRExcUnits[startTrialInd:endTrialInd])
frCorrEEBetweenUnits = np.corrcoef(R1.trialUpFRExcUnits[startTrialInd:endTrialInd].T)
frCorrEEBetweenTrials[np.triu_indices(frCorrEEBetweenTrials.shape[0])] = np.nan
frCorrEEBetweenUnits[np.triu_indices(frCorrEEBetweenUnits.shape[0])] = np.nan

In [277]:
frCorrEEBetweenUnitsAnti = frCorrEEBetweenUnits.copy()
frCorrEEBetweenUnitsAnti[R1.preEE, R1.posEE] = np.nan
print(np.nanmean(frCorrEEBetweenUnits))
print(np.nanmean(frCorrEEBetweenUnits[R1.preEE, R1.posEE]))
print(np.nanmean(frCorrEEBetweenUnitsAnti))

0.0018183634732697994
0.014625433244700269
-0.002451656570437842


In [213]:
# does the same type of bias exist in the correlations between FRs???

frCorrEEBetweenUnitsS = np.full(frCorrEEBetweenUnits.shape, np.nan)
frCorrEEBetweenUnitsS[R1.preEE, R1.posEE] = frCorrEEBetweenUnits[R1.preEE, R1.posEE]

allNumbers = frCorrEEBetweenUnits.ravel()

vlims = (-np.nanmax(np.fabs(frCorrEEBetweenUnits)), np.nanmax(np.fabs(frCorrEEBetweenUnits)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 7))
weight_matrix(ax7[0], frCorrEEBetweenUnitsS, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], frCorrEEBetweenUnits, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)

In [363]:
R1.trialUpFRInhUnits.shape

(6765, 400)

In [278]:
trialUpFRAllUnits = np.concatenate((R1.trialUpFRExcUnits, R1.trialUpFRInhUnits, ), axis=1)

In [379]:
trialUpFRAllUnits = np.concatenate((R2.trialUpFRExcUnits, R2.trialUpFRInhUnits, ), axis=1)

In [380]:
frCorrBetweenUnits = np.corrcoef(trialUpFRAllUnits[startTrialInd:endTrialInd].T)
frCorrBetweenUnits[np.triu_indices(frCorrBetweenUnits.shape[0])] = np.nan

frCorrEEBetweenUnits = frCorrBetweenUnits[:p['nExc'], :p['nExc']]
frCorrEIBetweenUnits = frCorrBetweenUnits[p['nExc']:, :p['nExc']]
frCorrIIBetweenUnits = frCorrBetweenUnits[p['nExc']:, p['nExc']:]

In [348]:
frCorrBetweenUnitsAnti = frCorrBetweenUnits.copy()
frCorrBetweenUnitsAnti[R1.preEE, R1.posEE] = np.nan
frCorrBetweenUnitsAnti[p['nExc'] + R1.preEI, R1.posEI] = np.nan
frCorrBetweenUnitsAnti[R1.preIE, p['nExc'] + R1.posIE] = np.nan
frCorrBetweenUnitsAnti[p['nExc'] + R1.preII, p['nExc'] + R1.posII] = np.nan

In [349]:
frCorrBetweenUnitsConn = np.full(frCorrBetweenUnits.shape, np.nan)
frCorrBetweenUnitsConn[R1.preEE, R1.posEE] = frCorrBetweenUnits[R1.preEE, R1.posEE]
frCorrBetweenUnitsConn[p['nExc'] + R1.preEI, R1.posEI] = frCorrBetweenUnits[p['nExc'] + R1.preEI, R1.posEI]
frCorrBetweenUnitsConn[R1.preIE, p['nExc'] + R1.posIE] = frCorrBetweenUnits[R1.preIE, p['nExc'] + R1.posIE]
frCorrBetweenUnitsConn[p['nExc'] + R1.preII, p['nExc'] + R1.posII] = frCorrBetweenUnits[p['nExc'] + R1.preII, p['nExc'] + R1.posII]

In [350]:
print(np.nanmean(frCorrBetweenUnits[:p['nExc'], :p['nExc']]))
print(np.nanmean(frCorrBetweenUnits[R1.preEE, R1.posEE]))

0.011065483134125077
0.011064017320842803


In [351]:
print(np.nanmean(frCorrBetweenUnits[p['nExc']:, :p['nExc']]))
print(np.nanmean(frCorrBetweenUnits[p['nExc'] + R1.preEI, R1.posEI]))
print(np.nanmean(frCorrBetweenUnits[p['nExc'] + R1.posIE, R1.preIE]))

0.012377046733391234
0.012418071920910939
0.012386202626964892


In [352]:
print(np.nanmean(frCorrBetweenUnits[p['nExc']:, p['nExc']:]))
print(np.nanmean(frCorrBetweenUnits[p['nExc'] + R1.preII, p['nExc'] + R1.posII]))

0.01198655592312059
0.01215968478355726


In [353]:
unitCorrEERavel = frCorrBetweenUnits[R1.preEE, R1.posEE]
unitCorrEIRavel = frCorrBetweenUnits[p['nExc'] + R1.preEI, R1.posEI]
unitCorrIERavel = frCorrBetweenUnits[p['nExc'] + R1.posIE, R1.preIE]
unitCorrIIRavel = frCorrBetweenUnits[p['nExc'] + R1.preII, p['nExc'] + R1.posII]

In [381]:
unitCorrEERavel = frCorrBetweenUnits[R2.preEE, R2.posEE]
unitCorrEIRavel = frCorrBetweenUnits[p['nExc'] + R2.preEI, R2.posEI]
unitCorrIERavel = frCorrBetweenUnits[p['nExc'] + R2.posIE, R2.preIE]
unitCorrIIRavel = frCorrBetweenUnits[p['nExc'] + R2.preII, p['nExc'] + R2.posII]

In [382]:
print(np.nanmean(unitCorrEERavel))
print(np.nanmean(unitCorrEIRavel))
print(np.nanmean(unitCorrIERavel))
print(np.nanmean(unitCorrIIRavel))

0.011065483134125082
0.012377046733391236
0.012377046733391238
0.01198655592312059


In [355]:
# correlate the unit correlation coefficient with the corresponding average cross-trial bias

In [356]:
(dwEEHHat[R1.preEE, R1.posEE], dwEIHHat[R1.preEI, R1.posEI], dwIEHHat[R1.preIE, R1.posIE], dwIIHHat[R1.preII, R1.posII], )

(array([ 0.01423898,  0.0110971 ,  0.1134997 , ...,  0.04314964,
         0.01663128, -0.18278544]),
 array([-0.39713344, -0.11542401,  0.06807905, ..., -0.3937676 ,
        -0.12531913, -0.29440433]),
 array([-0.14850719, -0.00151379, -0.35615783, ..., -0.09692234,
        -0.17748947, -0.22666331]),
 array([-0.20898584, -0.16637291, -0.19693493, ..., -0.45114331,
        -0.0202397 , -0.99746545]))

In [357]:
f, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

useAlpha = 1
useSize = 1

allX = np.concatenate((unitCorrEERavel, unitCorrEIRavel, unitCorrIERavel, unitCorrIIRavel, ))
allY = np.concatenate((dwEEHHat[R1.preEE, R1.posEE], dwEIHHat[R1.preEI, R1.posEI], dwIEHHat[R1.preIE, R1.posIE], dwIIHHat[R1.preII, R1.posII], ))

xLims = (np.nanmin(allX) - 0.1, np.nanmax(allX) + 0.1)
yLims = (np.nanmin(allY) - 0.1, np.nanmax(allY) + 0.1)

ax[0, 0].scatter(unitCorrEERavel, dwEEHHat[R1.preEE, R1.posEE], useSize, alpha=useAlpha, marker='.')
ax[0, 1].scatter(unitCorrEIRavel, dwEIHHat[R1.preEI, R1.posEI], useSize, alpha=useAlpha, marker='.')
ax[1, 0].scatter(unitCorrIERavel, dwIEHHat[R1.preIE, R1.posIE], useSize, alpha=useAlpha, marker='.')
ax[1, 1].scatter(unitCorrIIRavel, dwIIHHat[R1.preII, R1.posII], useSize, alpha=useAlpha, marker='.')

for anax in ax.ravel():
    anax.hlines(0, *xLims, linestyles='--')
    anax.vlines(0, *yLims, linestyles='--')
    
ax[0, 0].set(xlim=xLims, ylim=yLims, xlabel='connected E -> E corr. coeff.', ylabel='average proposed dwEE')
ax[0, 1].set(xlim=xLims, ylim=yLims, xlabel='connected E -> I corr. coeff.', ylabel='average proposed dwEI')
ax[1, 0].set(xlim=xLims, ylim=yLims, xlabel='connected I -> E corr. coeff.', ylabel='average proposed dwIE')
ax[1, 1].set(xlim=xLims, ylim=yLims, xlabel='connected I -> I corr. coeff.', ylabel='average proposed dwII')

[(-2.6635262096164136, 1.7732896704955854),
 Text(0, 0.5, 'average proposed dwII'),
 (-0.28957593935738385, 0.3196907665264049),
 Text(0.5, 0, 'connected I -> I corr. coeff.')]

In [386]:
# connected

f, ax = plt.subplots(figsize=(12, 4))

useBins = np.linspace(-0.2, 0.2, 100)
useDensity = True

ax.hist(unitCorrEERavel, bins=useBins, histtype='step', density=useDensity)
ax.hist(unitCorrEIRavel, bins=useBins, histtype='step', density=useDensity)
ax.hist(unitCorrIERavel, bins=useBins, histtype='step', density=useDensity)
ax.hist(unitCorrIIRavel, bins=useBins, histtype='step', density=useDensity)
ax.vlines(0, *ax.get_ylim())
ax.legend(('EE', 'EI', 'IE', 'II'))
ax.set(xlabel='Correlation Coefficient', ylabel='# of occurences')

[Text(0, 0.5, '# of occurences'), Text(0.5, 0, 'Correlation Coefficient')]

In [387]:
print(np.nanmean(frCorrBetweenUnits[:p['nExc'], :p['nExc']]))
print(np.nanmean(frCorrBetweenUnits[p['nExc']:, :p['nExc']]))
print(np.nanmean(frCorrBetweenUnits[p['nExc']:, :p['nExc']]))
print(np.nanmean(frCorrBetweenUnits[p['nExc']:, p['nExc']:]))

0.011065483134125077
0.012377046733391234
0.012377046733391234
0.01198655592312059


In [388]:
# all possible

f, ax = plt.subplots(figsize=(12, 4))

useBins = np.linspace(-0.2, 0.2, 100)
useDensity = True

ax.hist(frCorrBetweenUnits[:p['nExc'], :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax.hist(frCorrBetweenUnits[p['nExc']:, :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax.hist(frCorrBetweenUnits[p['nExc']:, :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax.hist(frCorrBetweenUnits[p['nExc']:, p['nExc']:].ravel(), bins=useBins, histtype='step', density=useDensity)
ax.vlines(0, *ax.get_ylim())
ax.legend(('EE', 'EI', 'IE', 'II'))
ax.set(xlabel='Correlation Coefficient', ylabel='# of occurences')

[Text(0, 0.5, '# of occurences'), Text(0.5, 0, 'Correlation Coefficient')]

In [389]:
# another way of showing it is to show it in an array of histograms

In [390]:
f, ax = plt.subplots(2, 2, figsize=(18, 8), sharex=True)

useBins = np.linspace(-0.15, 0.15, 100)
useDensity = False

ax[0, 0].hist(unitCorrEERavel, bins=useBins, histtype='step', density=useDensity)
ax[0, 1].hist(unitCorrEIRavel, bins=useBins, histtype='step', density=useDensity)
ax[1, 0].hist(unitCorrIERavel, bins=useBins, histtype='step', density=useDensity)
ax[1, 1].hist(unitCorrIIRavel, bins=useBins, histtype='step', density=useDensity)

ax[0, 0].hist(frCorrBetweenUnits[:p['nExc'], :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax[0, 1].hist(frCorrBetweenUnits[p['nExc']:, :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax[1, 0].hist(frCorrBetweenUnits[p['nExc']:, :p['nExc']].ravel(), bins=useBins, histtype='step', density=useDensity)
ax[1, 1].hist(frCorrBetweenUnits[p['nExc']:, p['nExc']:].ravel(), bins=useBins, histtype='step', density=useDensity)

ax[0, 0].set(title='EE', ylabel='# of occurences')
ax[0, 1].set(title='EI')
ax[1, 0].set(title='IE', ylabel='# of occurences', xlabel='Correlation Coefficient')
ax[1, 1].set(title='II', xlabel='Correlation Coefficient')

for anax in ax.ravel():
    anax.vlines(0, *anax.get_ylim(), linestyles='--')
    anax.legend(('connected', 'not connected'))

In [359]:
# correlation between E and I

vlims = (-np.nanmax(np.fabs(frCorrBetweenUnits)), np.nanmax(np.fabs(frCorrBetweenUnits)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 7), sharex=True, sharey=True)
weight_matrix(ax7[0], frCorrBetweenUnitsConn, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], frCorrBetweenUnits, xlabel='Post Index', ylabel='Pre Index',
              clabel='Corr. Coef.', limsMethod='custom', vlims=vlims)

In [51]:
dwIIHHatNoDiag = dwIIHHat.copy()
dwIIHHatNoDiag[np.diag_indices_from(dwIIHHatNoDiag)] = np.nan

In [170]:
doDensity = False

useMin = np.min([np.nanmin(dwIIHHatS), np.nanmin(dwIIHHatNoDiag),])
useMax = np.max([np.nanmax(dwIIHHatS), np.nanmax(dwIIHHatNoDiag),])

useBins = np.linspace(useMin, useMax, 100)

f, ax = plt.subplots()
ax.hist(dwIIHHatNoDiag[~np.isnan(dwIIHHatNoDiag)], bins=useBins, density=doDensity, histtype='step', color='g')
ax.hist(dwIIHHatS[~np.isnan(dwIIHHatS)], bins=useBins, density=doDensity, histtype='step', color='b')

ax.vlines(np.nanmean(dwIIHHatNoDiag), 0, ax.get_ylim()[1], colors='g', linestyle='--')
ax.vlines(np.nanmean(dwIIHHatS), 0, ax.get_ylim()[1], colors='b', linestyle='--')

ax.set(xlabel='Proposed dW', ylabel='# of occurences')
ax.legend(('all proposed dW', 'connected synapses'))

In [55]:
dwIIHNoDiag = dwIIH.copy()
dwIIHNoDiag[np.diag_indices_from(dwIIHNoDiag)] = np.nan

NameError: name 'dwIIH' is not defined

In [214]:
doDensity = False

useMin = np.min([np.nanmin(dwIIHS), np.nanmin(dwIIHNoDiag),])
useMax = np.max([np.nanmax(dwIIHS), np.nanmax(dwIIHNoDiag),])

useBins = np.linspace(useMin, useMax, 100)

f, ax = plt.subplots()
ax.hist(dwIIHS[~np.isnan(dwIIHS)], bins=useBins, density=doDensity, histtype='step', color='b')
ax.hist(dwIIHNoDiag[~np.isnan(dwIIHNoDiag)], bins=useBins, density=doDensity, histtype='step', color='g')
ax.vlines(np.nanmean(dwIIHS), 0, ax.get_ylim()[1], colors='b', linestyle='--')
ax.vlines(np.nanmean(dwIIHNoDiag), 0, ax.get_ylim()[1], colors='g', linestyle='--')
ax.legend(('actual', 'all'))

In [56]:
print(np.nanmean(dwIIHS))

NameError: name 'dwIIHS' is not defined

In [144]:
dwIIHHatSRandRavel = np.random.choice(dwIIHHat.ravel(), int(dwIIHHat.size / 4), replace=False)
dwIIHHatSRandRavel.mean()

0.06354718045386862

In [58]:
preInds, postInds = generate_adjacency_indices_within(p['nInh'], p['propConnect'], allowAutapses=False)
dwIIHHatSNewSyns = dwIIHHat[preInds, postInds]
dwIIHHatSNewSyns.mean()

0.039445272125656156

In [59]:
ravelIndsII = np.ravel_multi_index((R1.preII, R1.posII,), (p['nInh'], p['nInh']))

In [60]:
dwIIHHatSRavel = dwIIHHat.ravel()[ravelIndsII]
dwIIHHatSRavel.mean()

-0.21075704793662225

In [61]:
dwIIHHat.mean()

0.06896905171171298

In [62]:
np.nanmean(dwIIHHatNoDiag)

0.04081709116325066

In [63]:
np.nanmean(dwIIHHatS)

-0.21075704793662225

In [64]:
# now let's repeat the same thing as the fake, but the firing rates are correlated with each other (eccentric ellipse)

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSumF2 = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSumF2 = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSumF2 = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSumF2 = np.zeros((p['nInh'], p['nInh']))

commonRand = np.random.normal(0, 0.05, nTrials)
dummyInd = 0
for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = p['setUpFRExc'] + (commonRand[dummyInd] * p['setUpFRExc']) + np.random.normal(0, 0.1, p['nExc']) * p['setUpFRExc']
    movAvgUpFRInhUnits = p['setUpFRInh'] + (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dwEEHTrialSumF2 += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSumF2 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSumF2 += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSumF2 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits))
    dummyInd += 1

dwEEHHatF2 = dwEEHTrialSumF2 / nTrials
dwEIHHatF2 = dwEIHTrialSumF2 / nTrials
dwIEHHatF2 = dwIEHTrialSumF2 / nTrials
dwIIHHatF2 = dwIIHTrialSumF2 / nTrials

dwEEHHatSF2 = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatSF2 = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatSF2 = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatSF2 = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatSF2[R1.preEE, R1.posEE] = dwEEHHatF2[R1.preEE, R1.posEE]
dwEIHHatSF2[R1.preEI, R1.posEI] = dwEIHHatF2[R1.preEI, R1.posEI]
dwIEHHatSF2[R1.preIE, R1.posIE] = dwIEHHatF2[R1.preIE, R1.posIE]
dwIIHHatSF2[R1.preII, R1.posII] = dwIIHHatF2[R1.preII, R1.posII]

In [65]:
print(np.nanmean(dwEEHHatSF2) - dwEEHHatF2.mean())
print(np.nanmean(dwEIHHatSF2) - dwEIHHatF2.mean())
print(np.nanmean(dwIEHHatSF2) - dwIEHHatF2.mean())
print(np.nanmean(dwIIHHatSF2) - dwIIHHatF2.mean())

0.00019815192133687543
-0.00023176324764120904
0.0002886799915373939
-0.0037956936789795837


In [66]:
dwIIHHatNorm = dwIIHHat.copy()
dwIIHHatNorm[np.diag_indices_from(dwIIHHatNorm)] = 0
dwIIHHatNorm = dwIIHHatNorm / np.max(np.fabs(dwIIHHatNorm))

In [67]:
dwIIHHatNormF = dwIIHHatF.copy()
dwIIHHatNormF[np.diag_indices_from(dwIIHHatNormF)] = 0
dwIIHHatNormF = dwIIHHatNormF / np.max(np.fabs(dwIIHHatNormF))

In [68]:
dwIIHHatNormF2 = dwIIHHatF2.copy()
dwIIHHatNormF2[np.diag_indices_from(dwIIHHatNormF2)] = 0
dwIIHHatNormF2 = dwIIHHatNormF2 / np.max(np.fabs(dwIIHHatNormF2))

In [69]:
dwIIHHatNorm[:, 0]

array([ 0.00000000e+00,  3.30692436e-01,  5.85084199e-02, -3.25452263e-01,
       -1.10724434e-01, -9.33513766e-02,  2.93308681e-03,  1.40905829e-01,
        9.64936185e-02, -4.36818873e-02,  5.92230430e-02,  2.33087621e-01,
       -1.84758889e-01, -2.56458176e-01,  8.24481270e-02,  8.75811424e-02,
        5.34978521e-02, -1.50041197e-02, -4.15388107e-02,  9.48994939e-03,
        1.78297394e-01,  2.28724494e-01,  1.33324278e-01,  5.31235363e-02,
       -1.16529839e-01,  1.87293830e-01, -5.40561901e-02, -1.85866425e-01,
        6.77133913e-02,  5.32678735e-02,  2.33581667e-01,  3.97803748e-02,
       -1.59883331e-01,  3.46935795e-02,  2.20919580e-01, -2.84875679e-01,
       -1.29707955e-01,  5.52200177e-02, -8.67990751e-02, -9.48860949e-02,
        2.53980659e-01,  2.23057746e-01,  3.03130502e-01,  1.25172789e-02,
        4.58420660e-02,  2.79070225e-01,  3.58517338e-02, -2.33090827e-02,
       -1.04933439e-01,  1.65360077e-01, -8.76888034e-02, -2.52084854e-03,
        1.16524275e-01, -

In [70]:
dwIIHHatNormF[:, 0]

array([ 0.        , -0.12210316, -0.10983357, -0.11348636, -0.16553782,
       -0.08927872, -0.16647243, -0.12102108, -0.16857124, -0.13415019,
       -0.16098192, -0.09578992, -0.13953643, -0.11740248, -0.12627249,
       -0.14815868, -0.1136888 , -0.14710066, -0.17769469, -0.18775682,
       -0.11820397, -0.13354729, -0.15661426, -0.13057347, -0.10275234,
       -0.09295946, -0.13341465, -0.10266774, -0.07555564, -0.15202973,
       -0.17936197, -0.13182839, -0.17301157, -0.14783482, -0.15661527,
       -0.11911874, -0.14628531, -0.13669432, -0.14041829, -0.12441228,
       -0.12171915, -0.10138468, -0.1233399 , -0.15666706, -0.14206891,
       -0.11183099, -0.1600045 , -0.14207584, -0.15688646, -0.17577316,
       -0.16462439, -0.11757145, -0.1740296 , -0.13311107, -0.15574182,
       -0.14078519, -0.17512121, -0.11519818, -0.13488477, -0.10317022,
       -0.1731719 , -0.17312074, -0.08911369, -0.13098085, -0.18914832,
       -0.18461054, -0.16382215, -0.09964536, -0.15790161, -0.11

In [71]:
dwIIHHatNormF2[:, 0]

array([0.        , 0.30369244, 0.32587999, 0.27829854, 0.31742834,
       0.30903728, 0.32248871, 0.32784784, 0.29298362, 0.30596454,
       0.32183314, 0.35897506, 0.31451888, 0.33726393, 0.27690393,
       0.30161047, 0.28466908, 0.3197785 , 0.30960472, 0.29255071,
       0.32327243, 0.3233712 , 0.33075952, 0.30849937, 0.29992144,
       0.29385123, 0.3052381 , 0.31625632, 0.31141082, 0.29918697,
       0.32466444, 0.28597279, 0.32511386, 0.27964296, 0.31132363,
       0.28404704, 0.30748412, 0.31938209, 0.30274297, 0.32901424,
       0.26857891, 0.27541193, 0.34552504, 0.31238683, 0.29269919,
       0.32426437, 0.3587412 , 0.31050696, 0.34104168, 0.31792604,
       0.30672446, 0.33696597, 0.33044818, 0.30202037, 0.30287924,
       0.32926646, 0.31842036, 0.30714917, 0.33693087, 0.32879881,
       0.33662258, 0.32575031, 0.30399097, 0.30600226, 0.31603764,
       0.33026573, 0.2729713 , 0.33335014, 0.3093732 , 0.30523564,
       0.28259911, 0.27916346, 0.30169233, 0.32632577, 0.30115

In [76]:
# plotting the full and sparse versions of dwII

vlims = (-1, 1)

fig7, ax7 = plt.subplots(1, 3, figsize=(17, 5))
ax7 = ax7.ravel()
weight_matrix(ax7[0], dwIIHHatNorm, xlabel='Post Index', ylabel='Pre Index',
              clabel='', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIHHatNormF, xlabel='Post Index', ylabel='',
              clabel='', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[2], dwIIHHatNormF2, xlabel='Post Index', ylabel='',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [88]:
dwIIHHatNorm.mean()

0.015882438916602622

In [89]:
dwIIHHatNormF.mean()

0.009256578517511398

In [90]:
dwIIHHatNormF2.mean()

0.41346299913009016

In [91]:

mixUpFRs = False
normByIncomingConns = False
normByOutgoingConns = False
normByBothCH = False
normByBothH = False
normIncomingH = False
normHCheat = False
convergentInd = 400
deviation = 0.2
# E1 = R1.trialUpFRExc[convergentInd]
# E2 = R2.trialUpFRExc[convergentInd]
E = 6  # 4 or 6
I = 16.8  # 16.8 or 11.2
useIndex = 402

# movAvgUpFRExcUnits = np.ones((R1.p['nExc'])) * E * Hz
# movAvgUpFRInhUnits = np.ones((R1.p['nInh'])) * I * Hz

# movAvgUpFRExcUnits = np.linspace(p['setUpFRExc'] * 0.8, p['setUpFRExc'] * 1.2,  R1.p['nExc']) + 0.2 * p['setUpFRExc']
# movAvgUpFRInhUnits = np.linspace(p['setUpFRInh'] * 0.8, p['setUpFRInh'] * 1.2,  R1.p['nInh']) + 0.2 * p['setUpFRInh']

aEE = adjacency_matrix_from_flat_inds(p['nExc'], p['nExc'], R1.preEE, R1.posEE)
aIE = adjacency_matrix_from_flat_inds(p['nExc'], p['nInh'], R1.preIE, R1.posIE)
aEI = adjacency_matrix_from_flat_inds(p['nInh'], p['nExc'], R1.preEI, R1.posEI)
aII = adjacency_matrix_from_flat_inds(p['nInh'], p['nInh'], R1.preII, R1.posII)

nOutgoingToExcFromEachExc = aEE.sum(1)
nOutgoingToExcFromEachInh = aEI.sum(1)
nOutgoingToInhFromEachExc = aIE.sum(1)
nOutgoingToInhFromEachInh = aII.sum(1)

nIncomingExcOntoEachExc = aEE.sum(0)
nIncomingInhOntoEachExc = aEI.sum(0)
nIncomingExcOntoEachInh = aIE.sum(0)
nIncomingInhOntoEachInh = aII.sum(0)

if mixUpFRs:
    movAvgUpFRExcUnits[:int(R1.p['nExc'] / 3)] = p['setUpFRExc'] * (1 - deviation)
    movAvgUpFRExcUnits[int(R1.p['nExc'] / 3):] = p['setUpFRExc'] * (1 + deviation)
    movAvgUpFRInhUnits[:int(R1.p['nInh'] / 3)] = p['setUpFRInh'] * (1 - deviation)
    movAvgUpFRInhUnits[int(R1.p['nInh'] / 3):] = p['setUpFRInh'] * (1 + deviation)

movAvgUpFRExcUnits = R1.trialUpFRExcUnits[useIndex, :] * Hz
movAvgUpFRInhUnits = R1.trialUpFRInhUnits[useIndex, :] * Hz
    
# each of these is preInds elements long, so must be tiled by postInds
dwEECH1 = movAvgUpFRExcUnits * (p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz / Hz
dwEICH1 = -movAvgUpFRInhUnits * (p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz / Hz 
dwIECH1 = -movAvgUpFRExcUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz 
dwIICH1 = movAvgUpFRInhUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz 

# if normByOutgoingConns:
#     dwEECH1 *= (p['nExc'] * p['propConnect']) / nOutgoingToExcFromEachExc
#     dwEICH1 *= (p['nInh'] * p['propConnect']) / nOutgoingToExcFromEachInh
#     dwIECH1 *= (p['nExc'] * p['propConnect']) / nOutgoingToInhFromEachExc
#     dwIICH1 *= (p['nInh'] * p['propConnect']) / nOutgoingToInhFromEachInh

dwEECH = np.tile(dwEECH1.reshape(-1, 1), wEEInit2.shape[1])
dwEICH = np.tile(dwEICH1.reshape(-1, 1), wEIInit2.shape[1])
dwIECH = np.tile(dwIECH1.reshape(-1, 1), wIEInit2.shape[1])
dwIICH = np.tile(dwIICH1.reshape(-1, 1), wIIInit2.shape[1])

# norm by incoming AND outgoing
normMatEE = (p['nExc'] * p['nExc'] * p['propConnect'] ** 2) / np.outer(nOutgoingToExcFromEachExc, nIncomingExcOntoEachExc)
normMatEI = (p['nExc'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToExcFromEachInh, nIncomingInhOntoEachExc)
normMatIE = (p['nExc'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToInhFromEachExc, nIncomingExcOntoEachInh)
normMatII = (p['nInh'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToInhFromEachInh, nIncomingInhOntoEachInh) 

# norm by incoming

if normByOutgoingConns:
    dwEECH *= (p['nExc'] * p['propConnect']) / nOutgoingToExcFromEachExc.reshape(-1, 1)
    dwEICH *= (p['nInh'] * p['propConnect']) / nOutgoingToExcFromEachInh.reshape(-1, 1)
    dwIECH *= (p['nExc'] * p['propConnect']) / nOutgoingToInhFromEachExc.reshape(-1, 1)
    dwIICH *= (p['nInh'] * p['propConnect']) / nOutgoingToInhFromEachInh.reshape(-1, 1)

if normByIncomingConns:
    dwEECH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachExc).reshape(1, -1)
    dwEICH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachExc).reshape(1, -1)
    dwIECH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachInh).reshape(1, -1)
    dwIICH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachInh).reshape(1, -1)

if normByBothCH:
    dwEECH *= normMatEE
    dwEICH *= normMatEI
    dwIECH *= normMatIE
    dwIICH *= normMatII

# regular homeo (outer product)
# since outer strips units and because of alpha we multiply by Hz to convert to amps
dwEEH = np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
dwEIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
dwIEH = np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

if normByBothH:
    dwEEH *= normMatEE
    dwEIH *= normMatEI
    dwIEH *= normMatIE
    dwIIH *= normMatII

if normIncomingH:
    dwEEH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachExc).reshape(1, -1)
    dwEIH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachExc).reshape(1, -1)
    dwIEH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachInh).reshape(1, -1)
    dwIIH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachInh).reshape(1, -1)
    
if normHCheat:
    pass

dwEE = dwEECH + dwEEH
dwEI = dwEICH + dwEIH
dwIE = dwIECH + dwIEH
dwII = dwIICH + dwIIH

dwEES = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEIS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIES = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEES[R1.preEE, R1.posEE] = dwEE[R1.preEE, R1.posEE]
dwEIS[R1.preEI, R1.posEI] = dwEI[R1.preEI, R1.posEI]
dwIES[R1.preIE, R1.posIE] = dwIE[R1.preIE, R1.posIE]
dwIIS[R1.preII, R1.posII] = dwII[R1.preII, R1.posII]

dwEECHS = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEICHS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIECHS = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIICHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEECHS[R1.preEE, R1.posEE] = dwEECH[R1.preEE, R1.posEE]
dwEICHS[R1.preEI, R1.posEI] = dwEICH[R1.preEI, R1.posEI]
dwIECHS[R1.preIE, R1.posIE] = dwIECH[R1.preIE, R1.posIE]
dwIICHS[R1.preII, R1.posII] = dwIICH[R1.preII, R1.posII]

dwEEHS = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEIHS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIEHS = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEEHS[R1.preEE, R1.posEE] = dwEEH[R1.preEE, R1.posEE]
dwEIHS[R1.preEI, R1.posEI] = dwEIH[R1.preEI, R1.posEI]
dwIEHS[R1.preIE, R1.posIE] = dwIEH[R1.preIE, R1.posIE]
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [ ]:
# instead, use the "average" dW mat from the convergent period
dwII = dwIIHa

In [92]:
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [93]:
dwIIHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [94]:
dwIICH1 = movAvgUpFRInhUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz

In [95]:
dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

In [98]:
print(dwIICH.mean())
print(dwIIH.mean())
print(dwII.mean())
print(np.nanmean(dwIICHS))
print(np.nanmean(dwIIHS))
print(np.nanmean(dwIIS))
print(np.nanmean(dwIIS) - dwII.mean())
print(np.nanmean(dwIIHS) - dwIIH.mean())

0.4915015408736994
-1.5319528546712946
-1.040451313797595
0.49150088
-1.7381049
-1.2466041
-0.20615277117097186
-0.2061520847894598


In [147]:
dwIIH = dwIIHHat.copy()

In [148]:
dwIIHFlat = dwIIH.ravel()

In [149]:
dividerNums = np.random.rand(*dwIIHFlat.shape) * 4

dwIIHFlatSub1 = dwIIHFlat[np.logical_and(dividerNums >= 0, dividerNums < 1)]
dwIIHFlatSub2 = dwIIHFlat[np.logical_and(dividerNums >= 1, dividerNums < 2)]
dwIIHFlatSub3 = dwIIHFlat[np.logical_and(dividerNums >= 2, dividerNums < 3)]
dwIIHFlatSub4 = dwIIHFlat[np.logical_and(dividerNums >= 3, dividerNums < 4)]

diffSub1 = dwIIHFlatSub1.mean() - dwIIHFlat.mean()
diffSub2 = dwIIHFlatSub2.mean() - dwIIHFlat.mean()
diffSub3 = dwIIHFlatSub3.mean() - dwIIHFlat.mean()
diffSub4 = dwIIHFlatSub4.mean() - dwIIHFlat.mean()

print(diffSub1)
print(diffSub2)
print(diffSub3)
print(diffSub4)
print((diffSub1 + diffSub2 + diffSub3 + diffSub4) / 4)

-0.0037902051634822603
0.006684547698671828
0.0018275621013599391
-0.0047861885973650015
-1.607099020387373e-05


In [150]:
nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    diffArray[sampleInd] = np.random.choice(dwIIHFlat, int(dwIIHFlat.size * p['propConnect']), replace=False).mean() - dwIIH.mean()

print(diffArray.mean())

1.311847889555904e-05


In [151]:
dwEIHFlat = dwEIH.ravel()

nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    diffArray[sampleInd] = np.random.choice(dwEIHFlat, int(dwEIHFlat.size * p['propConnect']), replace=False).mean() - dwEIH.mean()

print(diffArray.mean())

0.0004420077314013161


In [152]:
# can't pick the diagonal...

probabilityArray = np.full((dwII.shape[0], dwII.shape[-1]), 1 / (dwII.shape[0] * (dwII.shape[-1] - 1)))
probabilityArray[np.diag_indices_from(probabilityArray)] = 0

nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    currentRandomSubset = np.random.choice(dwIIHFlat, int(dwIIHFlat.size * p['propConnect']), replace=False, p=probabilityArray.ravel())
    diffArray[sampleInd] = currentRandomSubset.mean() - dwIIH.mean()

print(diffArray.mean())

-0.028175957236761074


In [157]:
# choosing the same positions as from a different training session (basically equivalent to choosing randomly...)

targetFile3 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo-reMean_guessGoodWeights2e3p025LogNormal_reMean_2021-08-19-20-37_results'
R3 = Results()
R3.init_from_file(targetFile3, targetPath)

In [158]:
dwIIH[R3.preII, R3.posII].mean() - dwIIH.mean()

-0.021980929199457204

In [166]:
np.nanmean(dwIIHHatS) - np.nanmean(dwIIHHat)

-0.2797260996483352

In [153]:
# check the diagonals

diagonalInds = np.arange(-dwII.shape[0], dwII.shape[0])
diagonalMeans = np.zeros(*diagonalInds.shape)
diagonalMeansS = np.zeros(*diagonalInds.shape)

for diagonalInd, diagonalVal in enumerate(diagonalInds):
    diagonalMeans[diagonalInd] = np.diagonal(dwIIH, offset=diagonalVal).mean()
    diagonalMeansS[diagonalInd] = np.nanmean(np.diagonal(dwIIHS, offset=diagonalVal))

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
 [py.warnings]


In [108]:
f, ax = plt.subplots()
ax.plot(diagonalInds, diagonalMeansS)
ax.plot(diagonalInds, diagonalMeans)
ax.legend(('actual', 'full'))

In [154]:
np.nanmean(diagonalMeansS) - np.nanmean(diagonalMeans)

-1.717977756146144

In [110]:
dwIIHS.shape

(400, 400)

In [111]:
# check the rows / columns

rowMeans = dwIIH.mean(0)
rowMeansS = np.nanmean(dwIIHS, 0)

f, ax = plt.subplots()
ax.plot(np.arange(dwIIH.shape[0]), rowMeansS)
ax.plot(np.arange(dwIIH.shape[0]), rowMeans)
ax.legend(('actual', 'full'))

In [112]:
rowMeansS.mean() - rowMeans.mean()

-1.8615770744015323

In [113]:
rowMeansS.mean()

-1.792608

In [114]:
rowMeans.mean()

0.06896905171171298

In [115]:
# check the rows / columns

colMeans = dwIIH.mean(1)
colMeansS = np.nanmean(dwIIHS, 1)

f, ax = plt.subplots()
ax.plot(np.arange(dwIIH.shape[0]), colMeansS)
ax.plot(np.arange(dwIIH.shape[0]), colMeans)
ax.legend(('actual', 'full'))

In [116]:
colMeansS.mean() - colMeans.mean()

-1.8372813867260562

In [119]:
int(dwIIHFlat.size * p['propConnect'])

40000

In [120]:
dwIIHFlat.shape

(160000,)

In [121]:
print(dwEECH.mean())
print(dwEEH.mean())
print(dwEE.mean())
print(np.nanmean(dwEECHS))
print(np.nanmean(dwEEHS))
print(np.nanmean(dwEES))
print(np.nanmean(dwEES) - dwEE.mean())

0.5475677173443043
0.1756779759812953
0.7232456933256
0.54772836
0.15919223
0.70692027
-0.016325427174171736


In [122]:
print(dwEICH.mean())
print(dwEIH.mean())
print(dwEI.mean())
print(np.nanmean(dwEICHS))
print(np.nanmean(dwEIHS))
print(np.nanmean(dwEIS))
print(np.nanmean(dwEIS) - dwEI.mean())

-1.5319528546713177
-0.4915015408737082
-2.023454395545026
-1.5323205
-0.6117297
-2.1440504
-0.120595964180926


In [123]:
print(dwIECH.mean())
print(dwIEH.mean())
print(dwIE.mean())
print(np.nanmean(dwIECHS))
print(np.nanmean(dwIEHS))
print(np.nanmean(dwIES))
print(np.nanmean(dwIES) - dwIE.mean())

-0.17567797598129223
0.5475677173442958
0.3718897413630035
-0.17588355
0.47531742
0.29943398
-0.07245576495118405


In [167]:
# plotting the full and sparse versions of dwII

allNumbers = np.concatenate((dwII, dwIIS)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], dwII, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIS, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [176]:
# plotting the full and sparse versions of dwII - using the average post-convergence numbers for only the homeo portion

dwIIHHatPlot = dwIIHHat.copy()
dwIIHHatPlot[np.diag_indices_from(dwIIHHatPlot)] = np.nan

allNumbers = np.concatenate((dwIIHHatPlot, dwIIHHatS)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 7))
weight_matrix(ax7[0], dwIIHHatPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIHHatS, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [185]:
# plotting the sparse-INVERTED and ANTI-sparse versions of dwII - using the average post-convergence numbers for only the homeo portion

dwIIHHatSAntiPlot = dwIIHHatSAnti.copy()
dwIIHHatSAntiPlot[np.diag_indices_from(dwIIHHatSAntiPlot)] = np.nan

allNumbers = np.concatenate((dwIIHHatSAntiPlot, dwIIHHatSInv)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 7))
weight_matrix(ax7[0], dwIIHHatSAntiPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIHHatSInv, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [1110]:
# plotting the distribution of the weights that are kept in the sparse vs left out

In [125]:
dwEERemConn = dwEE.copy()
dwEIRemConn = dwEI.copy()
dwIERemConn = dwIE.copy()
dwIIRemConn = dwII.copy()

dwEERemConn[R1.preEE, R1.posEE] = np.nan
dwEIRemConn[R1.preEI, R1.posEI] = np.nan
dwIERemConn[R1.preIE, R1.posIE] = np.nan
dwIIRemConn[R1.preII, R1.posII] = np.nan

nBins = 100

f, ax = plt.subplots(2, 2)

ax[0, 0].hist(dwEE[R1.preEE, R1.posEE], bins=nBins, density=True, histtype='step')
ax[0, 1].hist(dwEI[R1.preEI, R1.posEI], bins=nBins, density=True, histtype='step')
ax[1, 0].hist(dwIE[R1.preIE, R1.posIE], bins=nBins, density=True, histtype='step')
ax[1, 1].hist(dwII[R1.preII, R1.posII], bins=nBins, density=True, histtype='step')

ax[0, 0].hist(dwEERemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[0, 1].hist(dwEIRemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[1, 0].hist(dwIERemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[1, 1].hist(dwIIRemConn.ravel(), bins=nBins, density=True, histtype='step')

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\si

(array([1.61904762e-06, 0.00000000e+00, 4.85714286e-06, 0.00000000e+00,
        0.00000000e+00, 6.47619048e-06, 1.13333333e-05, 1.61904762e-06,
        9.71428571e-06, 1.61904762e-05, 4.20952381e-05, 3.23809524e-06,
        5.01904762e-05, 5.18095238e-05, 1.61904762e-05, 7.60952381e-05,
        1.11714286e-04, 1.18190476e-04, 5.82857143e-05, 2.02380952e-04,
        1.79714286e-04, 2.07238095e-04, 2.93047619e-04, 3.78857143e-04,
        4.98666667e-04, 7.12380952e-04, 5.82857143e-04, 1.18676190e-03,
        1.40371429e-03, 1.14466667e-03, 2.23752381e-03, 2.16628571e-03,
        2.75076190e-03, 3.68657143e-03, 3.32876190e-03, 4.67257143e-03,
        5.00933333e-03, 4.61590476e-03, 7.62409524e-03, 7.75200000e-03,
        7.94466667e-03, 1.02971429e-02, 7.70342857e-03, 7.13028571e-03,
        8.90638095e-03, 1.45552381e-03, 1.11034286e-02, 3.91971429e-03,
        1.10370476e-02, 7.42980952e-03, 6.23009524e-03, 8.46276190e-03,
        5.66666667e-03, 7.82323810e-03, 4.67904762e-03, 4.656380

In [1173]:
dwIIHSRemConn = dwIIH.copy()
dwIIHSRemConn[R1.preII, R1.posII] = np.nan

useBins = np.linspace(dwIIH.min(), dwIIH.max(), 100)

f, ax = plt.subplots()
ax.hist(dwIIHS[R1.preII, R1.posII], bins=useBins, density=True, histtype='step')
ax.hist(dwIIHSRemConn.ravel(), bins=useBins, density=True, histtype='step')
ax.legend(('actual', 'all'))

In [1164]:
dwIICHSRemConn = dwIICH.copy()
dwIICHSRemConn[R1.preII, R1.posII] = np.nan

useBins = np.linspace(dwIICH.min(), dwIICH.max(), 100)

f, ax = plt.subplots()
ax.hist(dwIICHS[R1.preII, R1.posII], bins=useBins, density=True, histtype='step')
ax.hist(dwIICHSRemConn.ravel(), bins=useBins, density=True, histtype='step')
ax.legend(('actual', 'all'))

In [126]:
np.nanmean(dwIIS) - dwII.mean()

-0.20615277117097186

In [127]:
np.nanmean(dwIIHS) - dwIIH.mean()

-1.8070739911724674

In [128]:
np.nanmean(dwIICHS) - dwIICH.mean()

-6.565791895174478e-07

In [1060]:
plt.close('all')

In [985]:
# plotting the cross-homeo and homeo dW matrices next to each other

dwFullCH = np.block([[dwEECH, dwIECH], [-dwEICH, -dwIICH]])
dwFullH = np.block([[dwEEH, dwIEH], [-dwEIH, -dwIIH]])

allNumbers = np.concatenate((dwFullCH, dwFullH)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

wFullInitPlot = dwFullCH.copy()
wFullFinalPlot = dwFullH.copy()

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], wFullInitPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], wFullFinalPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [986]:
# plotting the two-term sum compared to its sparse version

dwFull = np.block([[dwEE, dwIE], [-dwEI, -dwII]])
dwFullS = np.block([[dwEES, dwIES], [-dwEIS, -dwIIS]])

allNumbers = np.concatenate((dwFull, dwFullS)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

wFullInitPlot = dwFull.copy()
wFullFinalPlot = dwFullS.copy()

wFullInitPlot[np.isnan(wFullInitPlot)] = 0
wFullFinalPlot[np.isnan(wFullFinalPlot)] = 0

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], wFullInitPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], wFullFinalPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [987]:
incomingExcCurrentOntoEachExcCH = np.nansum(dwEECHS, 0)
incomingInhCurrentOntoEachExcCH = np.nansum(dwEICHS, 0)
incomingExcCurrentOntoEachInhCH = np.nansum(dwIECHS, 0)
incomingInhCurrentOntoEachInhCH = np.nansum(dwIICHS, 0)
outgoingCurrentToExcFromEachExcCH = np.nansum(dwEECHS, 1)
outgoingCurrentToExcFromEachInhCH = np.nansum(dwEICHS, 1)
outgoingCurrentToInhFromEachExcCH = np.nansum(dwIECHS, 1)
outgoingCurrentToInhFromEachInhCH = np.nansum(dwIICHS, 1)

incomingExcCurrentOntoEachExcH = np.nansum(dwEEHS, 0)
incomingInhCurrentOntoEachExcH = np.nansum(dwEIHS, 0)
incomingExcCurrentOntoEachInhH = np.nansum(dwIEHS, 0)
incomingInhCurrentOntoEachInhH = np.nansum(dwIIHS, 0)
outgoingCurrentToExcFromEachExcH = np.nansum(dwEEHS, 1)
outgoingCurrentToExcFromEachInhH = np.nansum(dwEIHS, 1)
outgoingCurrentToInhFromEachExcH = np.nansum(dwIEHS, 1)
outgoingCurrentToInhFromEachInhH = np.nansum(dwIIHS, 1)

incomingExcCurrentOntoEachExc = np.nansum(dwEES, 0)
incomingInhCurrentOntoEachExc = np.nansum(dwEIS, 0)
incomingExcCurrentOntoEachInh = np.nansum(dwIES, 0)
incomingInhCurrentOntoEachInh = np.nansum(dwIIS, 0)
outgoingCurrentToExcFromEachExc = np.nansum(dwEES, 1)
outgoingCurrentToExcFromEachInh = np.nansum(dwEIS, 1)
outgoingCurrentToInhFromEachExc = np.nansum(dwIES, 1)
outgoingCurrentToInhFromEachInh = np.nansum(dwIIS, 1)

In [961]:
incomingExcCurrentOntoEachExcCH

array([-6713.287 , -6714.1016, -6713.2764, ..., -6704.177 , -6691.0044,
       -6704.8345], dtype=float32)

In [925]:
labels = ['incomingExcCurrentOntoEachExcCH', 'incomingInhCurrentOntoEachExcCH', 'incomingExcCurrentOntoEachInhCH', 'incomingInhCurrentOntoEachInhCH', 'outgoingCurrentToExcFromEachExcCH', 'outgoingCurrentToExcFromEachInhCH', 'outgoingCurrentToInhFromEachExcCH', 'outgoingCurrentToInhFromEachInhCH', 'incomingExcCurrentOntoEachExcH', 'incomingInhCurrentOntoEachExcH', 'incomingExcCurrentOntoEachInhH', 'incomingInhCurrentOntoEachInhH', 'outgoingCurrentToExcFromEachExcH', 'outgoingCurrentToExcFromEachInhH', 'outgoingCurrentToInhFromEachExcH', 'outgoingCurrentToInhFromEachInhH', 'incomingExcCurrentOntoEachExc', 'incomingInhCurrentOntoEachExc', 'incomingExcCurrentOntoEachInh', 'incomingInhCurrentOntoEachInh', 'outgoingCurrentToExcFromEachExc', 'outgoingCurrentToExcFromEachInh', 'outgoingCurrentToInhFromEachExc', 'outgoingCurrentToInhFromEachInh',]

In [926]:
allCVs = np.array([cv(incomingExcCurrentOntoEachExcCH), cv(incomingInhCurrentOntoEachExcCH), cv(incomingExcCurrentOntoEachInhCH), cv(incomingInhCurrentOntoEachInhCH), cv(outgoingCurrentToExcFromEachExcCH), cv(outgoingCurrentToExcFromEachInhCH), cv(outgoingCurrentToInhFromEachExcCH), cv(outgoingCurrentToInhFromEachInhCH), cv(incomingExcCurrentOntoEachExcH), cv(incomingInhCurrentOntoEachExcH), cv(incomingExcCurrentOntoEachInhH), cv(incomingInhCurrentOntoEachInhH), cv(outgoingCurrentToExcFromEachExcH), cv(outgoingCurrentToExcFromEachInhH), cv(outgoingCurrentToInhFromEachExcH), cv(outgoingCurrentToInhFromEachInhH), cv(incomingExcCurrentOntoEachExc), cv(incomingInhCurrentOntoEachExc), cv(incomingExcCurrentOntoEachInh), cv(incomingInhCurrentOntoEachInh), cv(outgoingCurrentToExcFromEachExc), cv(outgoingCurrentToExcFromEachInh), cv(outgoingCurrentToInhFromEachExc), cv(outgoingCurrentToInhFromEachInh), ])

In [927]:
cvDF = pd.DataFrame(allCVs * 100, labels)

In [928]:
cvDF

,0
incomingExcCurrentOntoEachExcCH,-0.418812
incomingInhCurrentOntoEachExcCH,0.837739
incomingExcCurrentOntoEachInhCH,0.423381
incomingInhCurrentOntoEachInhCH,-0.861423
outgoingCurrentToExcFromEachExcCH,-10.440950
outgoingCurrentToExcFromEachInhCH,10.383132
outgoingCurrentToInhFromEachExcCH,13.013013
outgoingCurrentToInhFromEachInhCH,-12.788823
incomingExcCurrentOntoEachExcH,-57.980984
incomingInhCurrentOntoEachExcH,58.653854


In [812]:
allCVs

array([-0.04245139,  0.08597401,  0.04286776, -0.08697999, -0.10442287,
        0.1038627 ,  0.13032407, -0.12737298, -0.57980984,  0.58653855,
       -0.5805523 ,  0.5879602 , -0.10687449,  0.10649461, -0.13842519,
        0.13689326, -0.15769655,  0.1769976 , -0.902717  ,  0.906475  ,
       -0.10449539,  0.10396127, -0.14992274,  0.1496405 ], dtype=float32)

In [528]:
print(incomingExcCurrentOntoEachExc.shape)

(1600,)


In [523]:
incomingExcCurrentOntoEachExc.std()

12.557544

In [524]:
outgoingExcCurrentToEachExc = np.nansum(dwEECHS, 1)
outgoingExcCurrentToEachExc

array([-6744.02  , -6709.878 , -6700.067 , ..., -6737.6187, -6722.0674,
       -6714.1416], dtype=float32)

In [525]:
outgoingExcCurrentToEachExc.std()

12.510517

In [526]:
np.nansum(dwEES, 0)

array([-9057.689, -8949.369, -9204.297, ..., -9171.056, -8984.055,
       -9165.846], dtype=float32)

In [527]:
np.nansum(dwEE, 0) / p['nExc']

array([-23.21856026, -24.24514108, -22.30134881, ..., -22.42062698,
       -23.67049097, -22.34091519])

In [224]:
np.nansum(dwEES, 0) / (~np.isnan(dwEES)).sum(0)

array([-22.79990959, -22.79991584, -22.799904  , ..., -22.79990473,
       -22.79991234, -22.79990424])

In [225]:
np.nansum(dwEES, 0) / (p['nExc'] * p['propConnect'])

array([-22.286911, -21.032923, -23.540901, ..., -23.369902, -21.716917,
       -23.483902], dtype=float32)

In [10]:
downSampleFactorForE = 0.025
downSampleFactorForI = 0.1

downSampleNForE = int(downSampleFactorForE * R1.p['nExc'])
downSampleNForI = int(downSampleFactorForI * R1.p['nInh'])

downSampleE = np.random.choice(R1.p['nExc'], size=downSampleNForE, replace=False)
downSampleI = np.random.choice(R1.p['nInh'], size=downSampleNForI, replace=False)

In [13]:
np.all(R1.trialUpFRExc == R2.trialUpFRExc)

True

In [14]:
np.all(R1.trialwEE == R2.trialwEE)

True

In [20]:
np.allclose(wFullInit1, wFullInit2, equal_nan=True)

True

In [21]:
np.allclose(wFullFinal1, wFullFinal2, equal_nan=True)

True

In [813]:
labels = ['incomingExcCurrentOntoEachExcCH',
'incomingInhCurrentOntoEachExcCH',
'incomingExcCurrentOntoEachInhCH',
'incomingInhCurrentOntoEachInhCH',
'outgoingCurrentToExcFromEachExcCH',
'outgoingCurrentToExcFromEachInhCH',
'outgoingCurrentToInhFromEachExcCH',
'outgoingCurrentToInhFromEachInhCH',
'incomingExcCurrentOntoEachExcH',
'incomingInhCurrentOntoEachExcH',
'incomingExcCurrentOntoEachInhH',
'incomingInhCurrentOntoEachInhH',
'outgoingCurrentToExcFromEachExcH',
'outgoingCurrentToExcFromEachInhH',
'outgoingCurrentToInhFromEachExcH',
'outgoingCurrentToInhFromEachInhH',
'incomingExcCurrentOntoEachExc',
'incomingInhCurrentOntoEachExc',
'incomingExcCurrentOntoEachInh',
'incomingInhCurrentOntoEachInh',
'outgoingCurrentToExcFromEachExc',
'outgoingCurrentToExcFromEachInh',
'outgoingCurrentToInhFromEachExc',
'outgoingCurrentToInhFromEachInh',]

In [810]:
incomingExcCurrentOntoEachExcCH
incomingInhCurrentOntoEachExcCH
incomingExcCurrentOntoEachInhCH
incomingInhCurrentOntoEachInhCH
outgoingCurrentToExcFromEachExcCH
outgoingCurrentToExcFromEachInhCH
outgoingCurrentToInhFromEachExcCH
outgoingCurrentToInhFromEachInhCH
incomingExcCurrentOntoEachExcH
incomingInhCurrentOntoEachExcH
incomingExcCurrentOntoEachInhH
incomingInhCurrentOntoEachInhH
outgoingCurrentToExcFromEachExcH
outgoingCurrentToExcFromEachInhH
outgoingCurrentToInhFromEachExcH
outgoingCurrentToInhFromEachInhH
incomingExcCurrentOntoEachExc
incomingInhCurrentOntoEachExc
incomingExcCurrentOntoEachInh
incomingInhCurrentOntoEachInh
outgoingCurrentToExcFromEachExc
outgoingCurrentToExcFromEachInh
outgoingCurrentToInhFromEachExc
outgoingCurrentToInhFromEachInh

-0.042451385
0.08597401
0.042867765
-0.086979985
-0.10442287
0.1038627
0.13032407
-0.12737298
-0.57980984
0.58653855
-0.5805523
0.5879602
-0.10687449
0.10649461
-0.13842519
0.13689326
-0.15769655
0.1769976
-0.902717
0.906475
-0.10449539
0.103961274
-0.14992274
0.1496405
